In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,RobustScaler

In [2]:
train_data= pd.read_csv('train_BRCpofr.csv',index_col=0)
test_data= pd.read_csv('test.csv',index_col=0)

In [3]:
# Removing duplicates
train_data = train_data.drop_duplicates()

In [4]:
train_data.head()

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,cltv
id,,,,,,,,,,,
1,Male,Urban,Bachelor,5L-10L,1,5,5790,More than 1,A,Platinum,64308
2,Male,Rural,High School,5L-10L,0,8,5080,More than 1,A,Platinum,515400
3,Male,Urban,Bachelor,5L-10L,1,8,2599,More than 1,A,Platinum,64212
4,Female,Rural,High School,5L-10L,0,7,0,More than 1,A,Platinum,97920
5,Male,Urban,High School,More than 10L,1,6,3508,More than 1,A,Gold,59736


In [5]:
test_data.head()

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy
id,,,,,,,,,,
89393,Female,Rural,High School,5L-10L,0,6,2134,More than 1,B,Silver
89394,Female,Urban,High School,2L-5L,0,4,4102,More than 1,A,Platinum
89395,Male,Rural,High School,5L-10L,1,7,2925,More than 1,B,Gold
89396,Female,Rural,Bachelor,More than 10L,1,2,0,More than 1,B,Silver
89397,Female,Urban,High School,2L-5L,0,5,14059,More than 1,B,Silver


In [6]:
target = train_data.pop('cltv')

In [7]:
def purchasing_power(x):
    
    # ratio is chosen based on general understanding and is not a fixed number
    #purchasing power is different for rural and urban income
    
    ratio =1.3
    if x['income']=='0-2L':
        if x['area']=='Rural':
            return 2
        else:
            return 2/ratio
    if x['income']=='2L-5L':
        if x['area']=='Rural':
            return 5
        else:
            return 5/ratio
        
    if x['income']=='5L-10L':
        if x['area']=='Rural':
            return 10
        else:
            return 10/ratio
        
    if x['income']=='More than 10L':
        if x['area']=='Rural':
            return 15
        else:
            return 15/ratio
        

In [8]:

def feature_engineering(df_received):
    
    df_transformed= df_received.copy()
    
    # changing only '<=2L' in income as it is not suitable for data column name
    map_income= {'5L-10L':'5L-10L', '2L-5L':'2L-5L', 'More than 10L':'More than 10L', '<=2L':'0-2L'}
    df_transformed['income']= df_transformed['income'].map(map_income)
    
    #purchasing power is different for rural and urban income
    df_transformed['purchasing_power']= df_transformed.apply(purchasing_power,axis=1)
    
    # changing marital status from numerical to categorical- useful for our scaling done later on
    map_marital= {1:'married',0:'not married'}
    df_transformed['marital_status']= df_transformed['marital_status'].map(map_marital)
    
    # created a new column that  tells about whether a person has made claim or not
    df_transformed['is_claimed']= df_transformed.apply(lambda x: 'yes' if x['claim_amount']>0 else 'no',axis=1)
    
    # earning to company has been created based on number of policies and how many year ago he bought his first policy
    df_transformed['earning_to_company']=df_transformed.apply(lambda x: x['vintage']*2.5 \
                                                   if x['num_policies']=='More than 1' else x['vintage'],axis=1)
    
    # claim amount per year can help us reduce effect of the number of years a person has been a member of policy
    df_transformed['claim_amount_per_year']=df_transformed['claim_amount']/(df_transformed.vintage +1)
    
    # it is done to help model differentiate between new cutomer and past customers
    df_transformed['new_customer'] = df_transformed.apply(lambda x:'yes' if x['vintage']<1 else 'no',axis=1)
    
    #claim per policy can reduce effect of number of policy on claims and is better measure
    df_transformed['claim_per_policy'] = df_transformed.apply(lambda x:x['claim_amount']/2 if x['num_policies']=='More than 1' \
                                                              else x['claim_amount'],axis=1)
    
    # to measure influence of corona on insurance habits
    df_transformed['during_corona'] = df_transformed.apply(lambda x:'yes' if x['vintage'] <=3 and x['vintage']>=1 else 'no',axis=1)
    
    return df_transformed
    

In [9]:
train_data = feature_engineering(train_data)
test_data = feature_engineering(test_data)

In [14]:
train_data.head()

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,purchasing_power,is_claimed,earning_to_company,claim_amount_per_year,new_customer,claim_per_policy,during_corona
id,,,,,,,,,,,,,,,,,
1,Male,Urban,Bachelor,5L-10L,married,5,5790,More than 1,A,Platinum,7.692308,yes,12.5,965.000000,no,2895.0,no
2,Male,Rural,High School,5L-10L,not married,8,5080,More than 1,A,Platinum,10.000000,yes,20.0,564.444444,no,2540.0,no
3,Male,Urban,Bachelor,5L-10L,married,8,2599,More than 1,A,Platinum,7.692308,yes,20.0,288.777778,no,1299.5,no
4,Female,Rural,High School,5L-10L,not married,7,0,More than 1,A,Platinum,10.000000,no,17.5,0.000000,no,0.0,no
5,Male,Urban,High School,More than 10L,married,6,3508,More than 1,A,Gold,11.538462,yes,15.0,501.142857,no,1754.0,no


In [15]:
test_data.head()

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,purchasing_power,is_claimed,earning_to_company,claim_amount_per_year,new_customer,claim_per_policy,during_corona
id,,,,,,,,,,,,,,,,,
89393,Female,Rural,High School,5L-10L,not married,6,2134,More than 1,B,Silver,10.000000,yes,15.0,304.857143,no,1067.0,no
89394,Female,Urban,High School,2L-5L,not married,4,4102,More than 1,A,Platinum,3.846154,yes,10.0,820.400000,no,2051.0,no
89395,Male,Rural,High School,5L-10L,married,7,2925,More than 1,B,Gold,10.000000,yes,17.5,365.625000,no,1462.5,no
89396,Female,Rural,Bachelor,More than 10L,married,2,0,More than 1,B,Silver,15.000000,no,5.0,0.000000,no,0.0,yes
89397,Female,Urban,High School,2L-5L,not married,5,14059,More than 1,B,Silver,3.846154,yes,12.5,2343.166667,no,7029.5,no


In [16]:
# scaling numerical variables
numerical_column= train_data.select_dtypes('number').columns

# robust scaling was used as our data do not have normal distribution
sc = RobustScaler()

train_data[numerical_column]= sc.fit_transform(train_data[numerical_column])

test_data[numerical_column]= sc.transform(test_data[numerical_column])

In [17]:
train_data.head()

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,purchasing_power,is_claimed,earning_to_company,claim_amount_per_year,new_customer,claim_per_policy,during_corona
id,,,,,,,,,,,,,,,,,
1,Male,Urban,Bachelor,5L-10L,married,0.000000,0.461382,More than 1,A,Platinum,0.000000,yes,0.50,0.240838,no,0.149203,no
2,Male,Rural,High School,5L-10L,not married,1.000000,0.268800,More than 1,A,Platinum,0.461538,yes,1.25,-0.197966,no,0.011626,no
3,Male,Urban,Bachelor,5L-10L,married,1.000000,-0.404150,More than 1,A,Platinum,0.000000,yes,1.25,-0.499957,no,-0.469118,no
4,Female,Rural,High School,5L-10L,not married,0.666667,-1.109107,More than 1,A,Platinum,0.461538,no,1.00,-0.816310,no,-0.972727,no
5,Male,Urban,High School,More than 10L,married,0.333333,-0.157591,More than 1,A,Gold,0.769231,yes,0.75,-0.267313,no,-0.292981,no


In [18]:
test_data.head()

,gender,area,qualification,income,marital_status,vintage,claim_amount,num_policies,policy,type_of_policy,purchasing_power,is_claimed,earning_to_company,claim_amount_per_year,new_customer,claim_per_policy,during_corona
id,,,,,,,,,,,,,,,,,
89393,Female,Rural,High School,5L-10L,not married,0.333333,-0.530277,More than 1,B,Silver,0.461538,yes,0.75,-0.482342,no,-0.559221,no
89394,Female,Urban,High School,2L-5L,not married,-0.333333,0.003526,More than 1,A,Platinum,-0.769231,yes,0.25,0.082430,no,-0.177881,no
89395,Male,Rural,High School,5L-10L,married,0.666667,-0.315725,More than 1,B,Gold,0.461538,yes,1.00,-0.415771,no,-0.405949,no
89396,Female,Rural,Bachelor,More than 10L,married,-1.000000,-1.109107,More than 1,B,Silver,1.461538,no,-0.25,-0.816310,no,-0.972727,yes
89397,Female,Urban,High School,2L-5L,not married,0.000000,2.704279,More than 1,B,Silver,-0.769231,yes,0.50,1.750606,no,1.751490,no


In [238]:
# Encoding our categorical variable by use of pandas dummy function
# 'drop first' is true as first column doesn't provide any extra information and can be derived from remaining column

train_data= pd.get_dummies(train_data,drop_first=True)
test_data= pd.get_dummies(test_data,drop_first=True)

In [239]:
# Adding 'cltv' back
train_data['cltv']= target

In [240]:
train_data.head()

,vintage,claim_amount,purchasing_power,earning_to_company,claim_amount_per_year,claim_per_policy,gender_Male,area_Urban,qualification_High School,qualification_Others,...,marital_status_not married,num_policies_More than 1,policy_B,policy_C,type_of_policy_Platinum,type_of_policy_Silver,is_claimed_yes,new_customer_yes,during_corona_yes,cltv
id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.461382,0.000000,0.50,0.240838,0.149203,1,1,0,0,...,0,1,0,0,1,0,1,0,0,64308
2,1.000000,0.268800,0.666667,1.25,-0.197966,0.011626,1,0,1,0,...,1,1,0,0,1,0,1,0,0,515400
3,1.000000,-0.404150,0.000000,1.25,-0.499957,-0.469118,1,1,0,0,...,0,1,0,0,1,0,1,0,0,64212
4,0.666667,-1.109107,0.666667,1.00,-0.816310,-0.972727,0,0,1,0,...,1,1,0,0,1,0,0,0,0,97920
5,0.333333,-0.157591,0.666667,0.75,-0.267313,-0.292981,1,1,1,0,...,0,1,0,0,0,0,1,0,0,59736


In [241]:
test_data.head()

,vintage,claim_amount,purchasing_power,earning_to_company,claim_amount_per_year,claim_per_policy,gender_Male,area_Urban,qualification_High School,qualification_Others,...,income_More than 10L,marital_status_not married,num_policies_More than 1,policy_B,policy_C,type_of_policy_Platinum,type_of_policy_Silver,is_claimed_yes,new_customer_yes,during_corona_yes
id,,,,,,,,,,,,,,,,,,,,,
89393,0.333333,-0.530277,0.666667,0.75,-0.482342,-0.559221,0,0,1,0,...,0,1,1,1,0,0,1,1,0,0
89394,-0.333333,0.003526,-0.666667,0.25,0.082430,-0.177881,0,1,1,0,...,0,1,1,0,0,1,0,1,0,0
89395,0.666667,-0.315725,0.666667,1.00,-0.415771,-0.405949,1,0,1,0,...,0,0,1,1,0,0,0,1,0,0
89396,-1.000000,-1.109107,1.666667,-0.25,-0.816310,-0.972727,0,0,0,0,...,1,0,1,1,0,0,1,0,0,1
89397,0.000000,2.704279,-0.666667,0.50,1.750606,1.751490,0,1,1,0,...,0,1,1,1,0,0,1,1,0,0


We have got a nice preprocessed data, let's save it for future use

In [242]:
train_data.to_csv('processed_train.csv')
test_data.to_csv('processed_test.csv')

In [243]:
pd.read_csv('processed_train.csv',index_col=0).head()

,vintage,claim_amount,purchasing_power,earning_to_company,claim_amount_per_year,claim_per_policy,gender_Male,area_Urban,qualification_High School,qualification_Others,...,marital_status_not married,num_policies_More than 1,policy_B,policy_C,type_of_policy_Platinum,type_of_policy_Silver,is_claimed_yes,new_customer_yes,during_corona_yes,cltv
id,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.461382,0.000000,0.50,0.240838,0.149203,1,1,0,0,...,0,1,0,0,1,0,1,0,0,64308
2,1.000000,0.268800,0.666667,1.25,-0.197966,0.011626,1,0,1,0,...,1,1,0,0,1,0,1,0,0,515400
3,1.000000,-0.404150,0.000000,1.25,-0.499957,-0.469118,1,1,0,0,...,0,1,0,0,1,0,1,0,0,64212
4,0.666667,-1.109107,0.666667,1.00,-0.816310,-0.972727,0,0,1,0,...,1,1,0,0,1,0,0,0,0,97920
5,0.333333,-0.157591,0.666667,0.75,-0.267313,-0.292981,1,1,1,0,...,0,1,0,0,0,0,1,0,0,59736
